# Optical Flow with Convolution Neural Networks

The paper's version of optical flow produces a vector <-x, +x, -y, +y, -z, +z>.
Our result vector needs to change in track_object(frame1, frame2, x, y, w, h, steps) - currently the displacement should be `[x,y]`, so we'll just need to do some conditional wizardry and expand it out.

*NOTE:* Prior to running any of this, a virtual environment is needed. Once in the venv, run `pip install -r requirements.txt` to prepare all of the required libraries.

In [1]:
import numpy as np
flow = np.array([-2,3])
final_flow = np.array([0, 0, 0, 0, 0, 0])

if flow[0] < 0:
    final_flow[0] = abs(flow[0])
elif flow[0] > 0:
    final_flow[1] = abs(flow[0])

if flow[1] < 0:
    final_flow[2] = abs(flow[1])
elif flow[1] > 0:
    final_flow[3] = abs(flow[1])
final_flow

array([2, 0, 0, 3, 0, 0])

The videos used are all releaseed under the Creative Commons Attribution 3.0 license.
We can generate the list of all files in the training directory (in this case, a scene from "Big Buck Bunny") as follows:

In [2]:
import os
file_list = os.listdir("bunny")

We'll need to add the directory name to the beginning of each file:

In [3]:
for x in range(len(file_list)):
    file_list[x] = 'bunny/' + file_list[x]

Preparing the dataset will use the `prepare_dataset` function in the `optical_flow.py` file. The signature for this function is:
`prepare_dataset(files_list, result_file, x, y, w, h, steps=5)`
The bounding box should be determined for the scene, and consists of the x,y,w,h parameters.

In [4]:
from optical_flow import prepare_dataset
prepare_dataset(file_list, "train-data.csv", 124, 158, 254, 233, steps=5)

To train the neural net, we'll use the Lkvf file.

In [5]:
from Lkvf import train
train()

Using TensorFlow backend.


Epoch 1/100
149/149 [==============================] - 0s 1ms/step - loss: 0.6884 - accuracy: 0.5996
Epoch 2/100
149/149 [==============================] - 0s 352us/step - loss: 0.6823 - accuracy: 0.7103
Epoch 3/100
149/149 [==============================] - 0s 360us/step - loss: 0.6769 - accuracy: 0.7740
Epoch 4/100
149/149 [==============================] - 0s 374us/step - loss: 0.6706 - accuracy: 0.8233
Epoch 5/100
149/149 [==============================] - 0s 381us/step - loss: 0.6664 - accuracy: 0.8333
Epoch 6/100
149/149 [==============================] - 0s 362us/step - loss: 0.6624 - accuracy: 0.8546
Epoch 7/100
149/149 [==============================] - 0s 340us/step - loss: 0.6569 - accuracy: 0.8803
Epoch 8/100
149/149 [==============================] - 0s 397us/step - loss: 0.6513 - accuracy: 0.8881
Epoch 9/100
149/149 [==============================] - 0s 392us/step - loss: 0.6498 - accuracy: 0.8881
Epoch 10/100
149/149 [==============================] - 0s 372us/step - los

149/149 [==============================] - 0s 348us/step - loss: 0.0706 - accuracy: 0.9821
Epoch 80/100
149/149 [==============================] - 0s 359us/step - loss: 0.0744 - accuracy: 0.9743
Epoch 81/100
149/149 [==============================] - 0s 370us/step - loss: 0.0517 - accuracy: 0.9866
Epoch 82/100
149/149 [==============================] - 0s 400us/step - loss: 0.0612 - accuracy: 0.9843
Epoch 83/100
149/149 [==============================] - 0s 312us/step - loss: 0.0663 - accuracy: 0.9843
Epoch 84/100
149/149 [==============================] - 0s 310us/step - loss: 0.0852 - accuracy: 0.9664
Epoch 85/100
149/149 [==============================] - 0s 317us/step - loss: 0.0590 - accuracy: 0.9855
Epoch 86/100
149/149 [==============================] - 0s 307us/step - loss: 0.0720 - accuracy: 0.9743
Epoch 87/100
149/149 [==============================] - 0s 324us/step - loss: 0.0543 - accuracy: 0.9799
Epoch 88/100
149/149 [==============================] - 0s 315us/step - loss:

Once we're trained up, we can test the neural network on arbitary video capture. We'll use the same video we trained on to try and generate a list of flow vectors.

In [6]:
from RunLKNN import run_network

run_network(file_list)

Loaded model from disk
[[0.00081155 0.00093774 0.00045903 0.00241075 0.00193895 0.00098933]]
[0.00012618199, 0.0019517198, -0.0009496234]
[[0.00107832 0.00128668 0.00067317 0.00318216 0.00248318 0.00134088]]
[0.0002083613, 0.0025089844, -0.0011422976]
[[0.00078478 0.00090585 0.00044204 0.00234557 0.001881   0.0009578 ]]
[0.000121068675, 0.0019035337, -0.00092319417]
[[0.00078841 0.00091097 0.00044424 0.00235418 0.00188968 0.00096234]]
[0.00012256199, 0.0019099426, -0.0009273376]
[[0.00074335 0.00085781 0.00041536 0.00225877 0.00179886 0.00091012]]
[0.00011446263, 0.0018434096, -0.00088874507]
[[0.00077482 0.00089931 0.00043441 0.00235971 0.00187537 0.000953  ]]
[0.00012449623, 0.0019252995, -0.00092236645]
[[0.00085336 0.00098008 0.00048648 0.00251744 0.00201538 0.00103779]]
[0.00012672675, 0.0020309614, -0.0009775853]
[[0.0007847  0.00090368 0.00044076 0.00234468 0.00187948 0.0009572 ]]
[0.00011898397, 0.0019039195, -0.0009222816]
[[0.00083579 0.00097407 0.0004783  0.00250503 0.001994

[[0.000833   0.00096371 0.00046846 0.00252132 0.00200057 0.00102451]]
[0.00013070839, 0.0020528608, -0.000976061]
[[0.00074829 0.00086519 0.00041769 0.00228192 0.00181359 0.00091847]]
[0.00011690572, 0.001864225, -0.0008951146]
[[0.00074354 0.00085893 0.00041532 0.00226327 0.00180047 0.00091136]]
[0.000115382834, 0.0018479531, -0.0008891133]
[[0.00084067 0.00097327 0.00047919 0.00248774 0.00199999 0.00102491]]
[0.00013259286, 0.002008546, -0.0009750783]
[[0.00075448 0.00086934 0.00042196 0.00228405 0.00182165 0.00092247]]
[0.00011485926, 0.0018620931, -0.0008991819]
[[0.00073912 0.00085513 0.00041204 0.00225853 0.00179491 0.00090757]]
[0.000116012525, 0.0018464875, -0.00088734576]
[[0.00075079 0.00086819 0.00041877 0.00229027 0.00182069 0.0009217 ]]
[0.000117396354, 0.0018714985, -0.0008989854]
[[0.00087265 0.00101102 0.00050021 0.00258704 0.00206783 0.00106654]]
[0.00013837742, 0.0020868327, -0.001001293]
[[0.00084212 0.0009781  0.0004843  0.00251762 0.00200244 0.00103185]]
[0.0001359

[[0.00075162 0.00086333 0.00042    0.00226168 0.00180939 0.00091558]]
[0.00011170813, 0.0018416804, -0.0008938094]
[[0.00073544 0.00085084 0.00041027 0.00224462 0.0017842  0.0009026 ]]
[0.00011539407, 0.0018343506, -0.00088159874]
[[0.00093377 0.00109095 0.00054653 0.00275313 0.00219946 0.00114544]]
[0.00015717972, 0.0022066017, -0.0010540113]
[[0.00080899 0.00093802 0.00045502 0.00244909 0.00194877 0.0009944 ]]
[0.00012903521, 0.0019940706, -0.00095437304]
[[0.00079236 0.00091857 0.00044517 0.00240034 0.00191078 0.00097298]]
[0.00012620853, 0.0019551672, -0.0009378005]
[[0.0010181  0.00121709 0.00062712 0.00305229 0.00237462 0.0012705 ]]
[0.0001989922, 0.0024251663, -0.0011041169]
[[0.00078569 0.0009055  0.00044275 0.00234495 0.00188158 0.00095788]]
[0.00011981203, 0.0019022087, -0.0009236964]
[[0.00112889 0.00135093 0.00071158 0.00331995 0.00258477 0.00140638]]
[0.00022204546, 0.0026083637, -0.0011783919]
[[0.00117119 0.00136763 0.00070828 0.0033623  0.00267712 0.00143211]]
[0.000196

[[0.00012618199, 0.0019517198, -0.0009496234],
 [0.0002083613, 0.0025089844, -0.0011422976],
 [0.000121068675, 0.0019035337, -0.00092319417],
 [0.00012256199, 0.0019099426, -0.0009273376],
 [0.00011446263, 0.0018434096, -0.00088874507],
 [0.00012449623, 0.0019252995, -0.00092236645],
 [0.00012672675, 0.0020309614, -0.0009775853],
 [0.00011898397, 0.0019039195, -0.0009222816],
 [0.00013828703, 0.0020267274, -0.0009670409],
 [0.00013031781, 0.002049112, -0.00097456714],
 [0.00013009092, 0.002048834, -0.0009741745],
 [0.00011541176, 0.0018377065, -0.0008828578],
 [0.00013091072, 0.0020128114, -0.0009807545],
 [0.00017831672, 0.0023831646, -0.0011036146],
 [0.00015842047, 0.0021572348, -0.0010133482],
 [0.00012779434, 0.0020208957, -0.00096230884],
 [0.00019472383, 0.0024088444, -0.0010934407],
 [0.00022596901, 0.002651988, -0.00119729],
 [0.00019281567, 0.002625095, -0.001225706],
 [0.000164948, 0.002265661, -0.0010595036],
 [0.00013736391, 0.0020820736, -0.0010016726],
 [0.00013850449, 0